In [1]:
#https://github.com/ekehoe32/orthrus
import sys
sys.path.append('/home/katrina/a/mankovic/ZOETIS/Fall2021/Orthrus/orthrus')
import orthrus
from orthrus import core
from orthrus.core import dataset
import numpy as np
import graph_tools_construction as gt
from matplotlib import pyplot as plt
import pandas
# from orthrus.core.pipeline import *
from sklearn.preprocessing import FunctionTransformer
from orthrus.preprocessing.imputation import HalfMinimum
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from orthrus.core.helper import load_object
from sklearn.preprocessing import StandardScaler
import os

import GLPE

# With Z75

In [2]:
pathway_data = pandas.read_csv('/data4/mankovic/ZOETIS/felis_catus_pathways.csv')
pathway_data = pathway_data.fillna(0)
pathway_data= pathway_data.rename(columns={"Unnamed: 0": "ReactomeID"})




In [3]:
ds = dataset.load_dataset(os.path.join('/data4/zoetis/Data/TPM_C1_Z34_Z40_Z42_Z75.ds'))
sample_ids  = ds.metadata['Project'] == 'Z75'
Z75_dataset = ds.slice_dataset(sample_ids=sample_ids)
preprocessing_transform = make_pipeline(HalfMinimum(missing_values=0), FunctionTransformer(np.log2), StandardScaler())
Z75_data = pandas.DataFrame(data = preprocessing_transform.fit_transform(Z75_dataset.data), columns = Z75_dataset.data.columns, index = Z75_dataset.data.index)


In [4]:
pathway_data = pathway_data[["ReactomeID"]+list(Z75_data.columns)]


In [5]:
new_column_dict = dict(zip(pathway_data.columns, ["ReactomeID"] + list(range(len(Z75_data.columns)))))
de_identified_pathway_data = pathway_data.rename(columns = new_column_dict)

In [6]:
better_pathway_data=pandas.DataFrame(columns = ['feature_id', 'pathway_id'])
gene_names = de_identified_pathway_data.columns
for row in np.array(de_identified_pathway_data):
    idx = np.where(row == 1)
    # print(row)
    for g in gene_names[idx]:
        better_pathway_data = better_pathway_data.append({'feature_id': int(g), 'pathway_id':row[0]}, ignore_index = True)


In [7]:
my_other_clpe = GLPE.CLPE('degree', 
                    'correlation',
                    better_pathway_data, 
                    heat_kernel_param = 2,
                    normalize_rows = False)
my_other_clpe

CLPE(centrality_measure='degree', heat_kernel_param=2.0,
     incidence_matrix=array([[3984, 'R-HSA-162699'],
       [3985, 'R-HSA-162699'],
       [3986, 'R-HSA-162699'],
       ...,
       [20877, 'R-HSA-5603037'],
       [26371, 'R-HSA-5603037'],
       [26376, 'R-HSA-5603037']], dtype=object),
     network_type='correlation', normalize_rows=0.0)

In [8]:
my_other_clpe.fit(np.array(Z75_data))

CLPE(centrality_measure='degree', heat_kernel_param=2.0,
     incidence_matrix=array([[3984, 'R-HSA-162699'],
       [3985, 'R-HSA-162699'],
       [3986, 'R-HSA-162699'],
       ...,
       [20877, 'R-HSA-5603037'],
       [26371, 'R-HSA-5603037'],
       [26376, 'R-HSA-5603037']], dtype=object),
     network_type='correlation', normalize_rows=0.0)

In [9]:
featureset = dataset.load_dataset('/data4/sharmak/zoetis/nate/z75_ssvm_feature_set_no_partitioning.pickle')
featureset_ids= list(featureset.index)

featureset_transition_matrix_ids = np.nonzero(np.in1d(np.array(Z75_data.columns),np.array(featureset_ids)))[0]
featureset_transition_matrix_ids

array([    3,    15,    18, ..., 28404, 28407, 28415])

In [10]:
simple_centrality_scores = my_other_clpe.simple_transform(featureset_transition_matrix_ids, n_null_trials = 10)

null trial 0 done


In [11]:
simple_centrality_scores 

,pathway,score,p_val
0,R-HSA-1059683,4.167885,0.0
1,R-HSA-109581,38.883818,0.0
2,R-HSA-109582,125.200471,0.0
3,R-HSA-109606,8.010259,0.1
4,R-HSA-109704,3.580888,0.8
...,...,...,...
1974,R-HSA-983695,11.465562,0.0
1975,R-HSA-983705,28.883333,0.0
1976,R-HSA-983712,29.311479,0.0
1977,R-HSA-991365,5.768671,0.0
